In [1]:
# | default_exp routes.card

In [2]:
# | exporti
import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de


In [3]:
#| export
class CardSearch_NotFoundError(de.DomoError):
    def __init__(self, card_id,
                 domo_instance, 
                 function_name,status,
                  message = None,
                 ):
        super().__init__(status=status,
                         message=f"card {card_id} not found", domo_instance=domo_instance, function_name=function_name)


In [4]:
# | export
class CardSearch_NotFoundError(de.DomoError):
    def __init__(
        self,
        card_id,
        domo_instance,
        function_name,
        status,
        parent_class: str = None,
        message=None,
    ):
        super().__init__(
            status=status,
            message=f"card {card_id} not found",
            domo_instance=domo_instance,
            function_name=function_name,
            parent_class=parent_class,
        )

In [ ]:
# | export
@gd.route_function
async def get_kpi_definition(
    auth: dmda.DomoAuth,
    card_id: str,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    parent_class: str = None,
    debug_num_stacks_to_drop=1,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/content/v3/cards/kpi/definition"

    body = {"urn": card_id}

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=body,
        debug_api=debug_api,
        session=session,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if not res.is_success and res.response == "Not Found":
        raise CardSearch_NotFoundError(
            card_id=card_id,
            status=res.status,
            domo_instance=auth.domo_instance,
            function_name="get_kpi_definition",
        )

    return res

#### sample implementation of get_card_kpi


In [5]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

card_id = 1766265020
await get_kpi_definition(card_id=card_id, auth=token_auth)

adjusting num_stacks_to_drop, consider revising `get_traceback` call
{16, 3, 12}


ResponseGetData(status=200, response={'id': 1766265020, 'urn': '1766265020', 'definition': {'subscriptions': {'big_number': {'name': 'big_number', 'columns': [{'column': 'Amount', 'aggregation': 'SUM', 'alias': 'Sum of Amount', 'format': {'type': 'abbreviated', 'format': '#'}}], 'filters': [], 'orderBy': [], 'groupBy': [], 'fiscal': False, 'projection': False, 'distinct': False, 'limit': 1}, 'main': {'name': 'main', 'columns': [{'column': 'Date', 'calendar': True, 'mapping': 'ITEM'}, {'column': 'Amount', 'aggregation': 'SUM', 'mapping': 'VALUE'}, {'column': 'Vendor', 'mapping': 'SERIES'}], 'filters': [], 'dateGrain': {'column': 'Date', 'dateTimeElement': 'DAY'}, 'orderBy': [], 'groupBy': [{'column': 'Vendor'}, {'column': 'Date', 'calendar': True}], 'fiscal': False, 'projection': False, 'distinct': False}}, 'formulas': [], 'conditionalFormats': [], 'annotations': [], 'slicers': [], 'title': 'Vendor Amounts Dates', 'description': '', 'chartVersion': '11', 'charts': {'main': {'component':

In [6]:
# | export
@gd.route_function
async def get_card_metadata(
    auth: dmda.DomoAuth,
    card_id: str,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    parent_class: str = None,
    debug_num_stacks_to_drop=1,
) -> rgd.ResponseGetData:
    optional_params = "metadata,certification,datasources,owners,problems"

    url = f"https://{auth.domo_instance}.domo.com/api/content/v1/cards?urns={card_id}&parts={optional_params}"

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        debug_api=debug_api,
        session=session,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if res.is_success and len(res.response) == 0:
        raise CardSearch_NotFoundError(
            card_id=card_id,
            status=res.status,
            domo_instance=auth.domo_instance,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    res.response = res.response[0]

    return res

#### sample implementation of get_card_metadata


In [7]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

card_id = 1766265020
res = await get_card_metadata(card_id=card_id, auth=token_auth)

res.response

adjusting num_stacks_to_drop, consider revising `get_traceback` call
{16, 3, 12}
ResponseGetData(status=200, response={'metadata': {'chartType': 'badge_vert_stackedbar', 'chartVersion': '7', 'currentLabel': 'Sum of Amount', 'historyId': 'b9e50cd4-19c8-44f2-a44c-6c72bd432125', 'allTime': '{"groupName":"Other","name":"DEFAULT","type":"Other","selected":true,"estimatedSeconds":-1,"dateLabel":"","column":"Date"}', 'calendar': 'default', 'columnAliases': '{}', 'columnFormats': '{}', 'defaultDateGrain': 'Day', 'SummaryNumberFormat': '{"type":"number","format":"#A"}'}, 'owners': [{'id': '1893952720', 'type': 'USER', 'displayName': 'Jae Wilson1'}], 'datasources': [{'dataSourceId': 'fd70d83b-38dc-40a2-9349-357ec4321d3e', 'dataSourceName': 'Vendor Amounts Dates', 'displayType': 'webform', 'dataType': 'webform', 'providerType': 'webform', 'isSampleData': False, 'lastUpdated': 1603767029732, 'adc': False, 'phase': None, 'state': 'SUCCESS'}], 'certification': {'state': 'NOT_CERTIFIED', 'adminCertif

In [8]:
# | export
def generate_body_search_cards_admin_summary(
    page_ids: [str] = None,
    #  searchPages: bool = True,
    card_search_text: str = None,
    page_search_text: str = None,
) -> dict:
    body = {"ascending": True, "orderBy": "cardTitle"}

    if card_search_text:
        body.update(
            {"cardTitleSearchText": card_search_text, "includeCardTitleClause": True}
        )

    if page_search_text:
        body.update(
            {
                "pageTitleSearchText": page_search_text,
                "includePageTitleClause": True,
                "notOnPage": False,
            }
        )

    if page_ids:
        body.update({"pageIds": page_ids})

    return body

In [9]:
# | export
@gd.route_function
async def search_cards_admin_summary(
    auth: dmda.DomoAuth,
    body: dict,
    maximum: int = None,
    debug_api: bool = False,
    debug_loop: bool = False,
    session: httpx.AsyncClient = None,
    wait_sleep: int = 3,
    parent_class: str = None,
    debug_num_stacks_to_drop: int = 1,
) -> rgd.ResponseGetData:
    limit = 100
    offset = 0
    loop_until_end = False if maximum else True

    url = f"https://{auth.domo_instance}.domo.com/api/content/v2/cards/adminsummary?skip={offset}&limit={limit}"

    offset_params = {
        "offset": "skip",
        "limit": "limit",
    }

    def arr_fn(res):
        return res.response.get("cardAdminSummaries")

    res = await gd.looper(
        auth=auth,
        method="POST",
        url=url,
        arr_fn=arr_fn,
        offset_params=offset_params,
        limit=limit,
        skip=offset,
        body=body,
        maximum=maximum,
        session=session,
        debug_api=debug_api,
        debug_loop=debug_loop,
        loop_until_end=loop_until_end,
        wait_sleep=wait_sleep,
        parent_class=parent_class,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
    )
    return res

#### sample implementation of search_cards_admin_summary


In [10]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

body = generate_body_search_cards_admin_summary()

print(body)

card_id = 1766265020
res = await search_cards_admin_summary(
    body=body,
    maximum=5,
    auth=token_auth,
    debug_api=False,
    debug_loop=False,
    wait_sleep=5,
)

res.response[0:5]

{'ascending': True, 'orderBy': 'cardTitle'}


[{'owners': [{'id': '722128535',
    'type': 'USER',
    'displayName': 'Miguel Guedes'}],
  'id': 332819959,
  'type': 'kpi',
  'badgeUpdated': 1632747565000,
  'title': ' Forecast-Sales_MG',
  'locked': False,
  'pageHierarchy': [{'pageId': -100000, 'title': 'Overview'}]},
 {'owners': [{'id': '583215149',
    'type': 'USER',
    'displayName': 'John Jarnagin'}],
  'id': 1582016299,
  'type': 'document',
  'badgeUpdated': 1693333789000,
  'title': " Incorrect Append - Forecast Column Header Names Don't Match",
  'locked': False,
  'pageHierarchy': [{'pageId': 237406695,
    'title': 'Session 2: Data Transformation',
    'parentPageId': 1374726359,
    'parentPageTitle': 'NPO | Sept 2023 Cohort'},
   {'pageId': 2037415261,
    'title': 'Session 2: Data Transformation',
    'parentPageId': 307648701,
    'parentPageTitle': 'Domo Customer Orientation  (DCO)'}]},
 {'owners': [{'id': '1251168784',
    'type': 'USER',
    'displayName': 'Andre Wieczorkiewicz'}],
  'id': 231472916,
  'type':

In [11]:
# | hide
import nbdev

nbdev.nbdev_export()

!nbqa black card.ipynb